In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import joblib

In [3]:
import tensorflow as tf

In [4]:
class Parameters():
    def __init__(self):
        self.description = 'Model for time-series forecasting'

        # Input sequence length - look-back
        self.Lag = 64
        # Prediction sequence length
        self.Horizon = 14
        # Trained model path
        self.model_path = 'S_1.h5'

        # Filename
        self.test_file = './S_test.csv'
        # Target series name
        self.targetSeries = 'PM25_Concentration'

        self.scalers_filename = "S.joblib"

args = Parameters()

In [5]:
# Start timer
#
start = time.time()

# Load data
#
df_test = pd.read_csv( args.test_file )

print('[INFO] Data imported')

df_test["Unnamed: 0"] = df_test["Unnamed: 0"].astype('datetime64')
df_test.set_index("Unnamed: 0", inplace=True)
df_test.head(3)

[INFO] Data imported


,weather,temperature,pressure,humidity,wind_speed,wind_direction,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration,PM25_Concentration
Unnamed: 0,,,,,,,,,,,,
2015-03-25 06:00:00,2.0,0.570588,0.631148,0.76,0.013889,13.0,0.050505,0.029915,0.041591,0.226158,0.010020,24.0
2015-03-25 07:00:00,2.0,0.570588,0.639344,0.75,0.013889,13.0,0.074495,0.064103,0.042851,0.182561,0.012024,30.0
2015-03-25 08:00:00,2.0,0.570588,0.639344,0.75,0.013889,13.0,0.074495,0.064103,0.042851,0.182561,0.012024,30.0


In [6]:
scalers = joblib.load(args.scalers_filename)

for feature, scaler in scalers.items():
    df_test[feature] = scaler.transform(df_test[feature].to_numpy().reshape(-1, 1))

In [7]:
def create_dataset(df, targetSeries, Lag = 1, Horizon = 1):

    dataX, dataY, dataDate = [], [], []
    # dfX = df.drop(targetSeries, axis=1)
    dfY = df[targetSeries]

    for i in tqdm( range(0, df.shape[0] + 1  - Lag - Horizon) ):

        dataX.append( df.to_numpy()[i:(i+Lag)] )
        dataY.append( dfY.to_numpy()[i + Lag : i + Lag + Horizon] )
        dataDate.append( df.index[i + Lag : i + Lag + Horizon].tolist() )

    return ( np.array(dataX), np.array(dataY), np.array(dataDate) )

In [8]:
testX, testY, testDate = create_dataset(df = df_test,
                                          Lag = args.Lag,
                                          Horizon = args.Horizon,
                                          targetSeries = args.targetSeries)

testY  = np.expand_dims(testY,  axis = -1)

print('\nTesting data shape:    ', testX.shape,  testY.shape)

100%|██████████| 781/781 [00:00<00:00, 17023.17it/s]


Testing data shape:     (781, 64, 12) (781, 14, 1)


In [9]:
model = tf.keras.models.load_model(args.model_path)

In [10]:
loss, mae = model.evaluate(testX, testY)
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)

25/25 [==============================] - 1s 16ms/step - loss: 0.0063 - mae: 0.0604
Mean Square Error: 0.006272353697568178
Mean Absolute Error: 0.060387324541807175
